# Plot boundaries on the Earth

Ce notebook est un résumé du notebook situé ici 

https://chart-studio.plotly.com/~empet/14813/heatmap-plot-on-a-spherical-map/#/

et se concentre sur le traçage des frontières (plus généralement d'un polygone) sur la sphère.

Il nécessite l'installation de **mpl_toolkits** qui peut être assez complexe.

Sous linux, sous réserve que la librairie GEOS soit installée, il suffit d'effectuer les commandes suivantes (le dossier à télécharger est assez lourd et la procédure peut prendre quelques minutes) :

En cas de problème, voir les détails ici :

https://matplotlib.org/basemap/users/installing.html .


In [6]:
import numpy as np           
from mpl_toolkits.basemap import Basemap
import plotly.graph_objects as go
import IPython
from numpy import pi, cos, sin
import matplotlib.pyplot as plt

In [7]:

def d2r(x):
    return x * np.pi / 180.

def mapping_map_to_sphere(lon, lat, radius=6371.):
    #this function maps the points of coords (lon, lat) to points onto the  sphere of radius radius
    
    lon=np.array(lon, dtype=np.float64)
    lat=np.array(lat, dtype=np.float64)
    lon=d2r(lon)
    lat=d2r(lat)
    xs=radius*cos(lon)*cos(lat)
    ys=radius*sin(lon)*cos(lat)
    zs=radius*sin(lat)
    return xs, ys, zs


# Functions converting coastline/country polygons to lon/lat traces
def polygons_to_traces(poly_paths, N_poly):
    ''' 
    pos arg 1. (poly_paths): paths to polygons
    pos arg 2. (N_poly): number of polygon to convert
    '''
    # init. plotting list
    lons=[]
    lats=[]

    for i_poly in range(N_poly):
        poly_path = poly_paths[i_poly]
        
        # get the Basemap coordinates of each segment
        coords_cc = np.array(
            [(vertex[0],vertex[1]) 
             for (vertex,code) in poly_path.iter_segments(simplify=False)]
        )
        
        # convert coordinates to lon/lat by 'inverting' the Basemap projection
        lon_cc, lat_cc = m(coords_cc[:,0],coords_cc[:,1], inverse=True)
    
        
        lats.extend(lat_cc.tolist()+[None]) 
        lons.extend(lon_cc.tolist()+[None])
        
       
    return lons, lats


# Function generating coastline lon/lat 
def get_coastline_traces():
    poly_paths = m.drawcoastlines().get_paths() # coastline polygon paths
    N_poly = 91  # use only the 91st biggest coastlines (i.e. no rivers)
    cc_lons, cc_lats= polygons_to_traces(poly_paths, N_poly)
    return cc_lons, cc_lats

# Function generating country lon/lat 
def get_country_traces():
    poly_paths = m.drawcountries().get_paths() # country polygon paths
    N_poly = len(poly_paths)  # use all countries
    country_lons, country_lats= polygons_to_traces(poly_paths, N_poly)
    return country_lons, country_lats








In [8]:
# Make shortcut to Basemap object, 
# not specifying projection type for this example
m = Basemap() 



# Get list of of coastline, country, and state lon/lat 

plt.figure()
cc_lons, cc_lats=get_coastline_traces()
country_lons, country_lats=get_country_traces()
plt.close()
#concatenate the lon/lat for coastlines and country boundaries:
lons=cc_lons+[None]+country_lons
lats=cc_lats+[None]+country_lats


xs, ys, zs=mapping_map_to_sphere(np.array(lons),np.array(lats), radius=6371.)# here the radius is slightly greater than 1 
                                                         #to ensure lines visibility; otherwise (with radius=1)
                                                         # some lines are hidden by contours colors

        
boundaries=dict(type='scatter3d',
               x=xs,
               y=ys,
               z=zs,
               mode='lines',
               line=dict(color='black', width=1)
              )

fig = go.Figure(data = [boundaries]
               )
fig.show()